### Patent annotation:  A novel dataset to highlight patent passages

#### Types of data samples

- *neutral_samples*: Here the paragraphs just above 'Advantageous Effects of Invention' are collected. 
   To have a nearly uniformed sequence lengths, only 3 paragraphs are collected. 

- *positive_samples*: All paragraphs from section/heading 'Advantageous Effects of Invention'
	are collected.

- *negative samples*: Paragraphs under the section 'Technical Problem' are collected. 


- Some of the examples patents containing interested tags as mentioned above are: (just for reference)
- You can open Google patents in advanced mode and search for below patents and look for above tags. 
    - US10842211B2
    - US10842310B2
    - US10842344B2 
	


### Get the statistics

Raw data can be downloaded from: https://bulkdata.uspto.gov/data/patent/grant/redbook/fulltext/2020/ 

1. Traverse USPTO bulk data files directory. 
2. Each xml file contains patents (nearly 8k) granted per week. Essentially, there are 52 such xml files per every year. Each such xml file is a nested file, such that it contains nearly 8k patents and each patent is embedded in xml file.
3. Per each week, find the total number of patents and total number of pos/neg/neutral samples found and write these statistics to a csv file. 

#### For instance:

- Year 2020 contains week_1.xml, week_2.xml, ...week_52.xml (total 52 xml files)
- week_1.xml is nested file, means to say, it further contains 8k .xml files, each one for a patent. 

#### Download the files from USPTO:
- use this url: https://bulkdata.uspto.gov/data/patent/grant/redbook/fulltext/2020/ 
- save it to a directory, for instance: "/home/renuk/Documents/USPTO_BULK_DATA/patents/2020/"
- Now you can execute the below codes. 

In [ ]:
def stats(filename):
    #home_dir = "/home/renuk/Documents/USPTO_BULK_DATA/patents/2020/"
    #file_path =home_dir+filename
    
    total_neutral_samples = 0 #total_positive_samples , total_negative_samples, total_neutral_samples
    total_publications = 0
    xml_text = html.unescape(open(filename, 'r').read())
    weekly_filename = filename.split('/')
    weekly_filename = weekly_filename[-1]
    print(weekly_filename)
    for patent in xml_text.split("<?xml version=\"1.0\" encoding=\"UTF-8\"?>"):

        if patent is None or patent == "":
            continue
        patent_text = patent
    
        bs = BeautifulSoup(patent, "lxml")
            
                #try:
                
        fwu_neutral = bs.find('heading',text='Solution to Problem') #fwu_advantages, fwu_problems, fwu_neutral
    
        #Advantageous Effects of Invention --- for positive samples
        #Technical Problem --- for negative samples
        #Solution to Problem --- for neutral samples, also the paragraphs (2/3) which are just above the 'Advantageous Effects of Invention' section can be used for neural samples 
        
        
        
        
        
        
    ##### search for application or grant
        application = bs.find('us-patent-application')
        if application is None: # If no application, search for grant
            application = bs.find('us-patent-grant')
        
        title = "None"
            
    #### Search for its title
            
        try:
            title = application.find('invention-title').text
        except Exception as e:          
            #print("no title found")
            title = ""
                
        #print("patent is:", title)
            
            
    #### Search for publication number
        try:
            #publication_country = application.find('publication-reference').find('country').text
            #publication_doc_number = application.find('publication-reference').find('doc-number').text
            publication_kind = application.find('publication-reference').find('kind').text
            #publication_num = publication_country+publication_doc_number+publication_kind
        except Exception as e:
            #publication_num = ""
            publication_kind = ""
        
        
        try:
            publication_num = application['file'].split("-")[0]
        
        except Exception as e:
            publication_num = ""
        
        
        publication_num = publication_num+publication_kind
        #print(publication_num)  
    
        try:
            application_type = application.find('application-reference')['appl-type']
        
        except Exception as e:
            application_type =''
        #print(application_type)
            
        if publication_num:
            total_publications +=1

        if fwu_neutral:  #fwu_advantages, fwu_problems, fwu_neutral
            total_neutral_samples +=1
        
    
 
    return weekly_filename, total_publications, total_neutral_samples
   
 

In [ ]:
import time

import pprint
import os
import sys
import html
import datetime
import pandas as pd
from bs4 import BeautifulSoup
import re
from datetime import datetime



path = "/home/renuk/Documents/USPTO_BULK_DATA/patents/2020/" #directory containing weekly xml files (i.e., 52 files per year)
all_files = glob.glob(os.path.join(path, "ipg*.xml"))
df = pd.DataFrame(columns=['weekly_filename', 'total_publications', 'total_neutral_samples'])
for filename in all_files:
    weekly_filename, total_publications, total_neutral_samples = stats(filename)
    df = df.append({'weekly_filename':weekly_filename, 'total_publications': total_publications,
                                'total_neutral_samples':total_neutral_samples},ignore_index=True)
df.to_csv("/home/renuk/Documents/USPTO_BULK_DATA/patents/2020_neutral_stats.csv")

### Traverse 52 xml files and access postive, negative and neutral samples. 

In [ ]:
# import the following 
import os, glob
import pandas as pd

import time

import pprint
import os
import sys
import html
import datetime
import pandas as pd
from bs4 import BeautifulSoup
import re
from datetime import datetime

#### Note:
- change the name of 4th coloumn according to your sample of interest. For instance: 'positive_text' for +ve label.
- change tags also accordingly. For instance: fwu_advantages = bs.find('heading',text='Advantageous Effects of Invention') for +ve labels
- go through each line of code and change the variable name wherever necessary.
- Below code traverse each weekly xml file, find the labels, if found then it will collect the pargarphs from that particular section or tag. 
- For every weekly xml, there is seperate csv file is stored with data automatically. For instance: '/home/renuk/Documents/USPTO_BULK_DATA/patents/2020/ipg200211.csv'

In [ ]:
def get_xml_data(filename):
    #home_dir = "/home/renuk/Documents/USPTO_BULK_DATA/patents/2020/"
    #file_path =home_dir+filename
    df = pd.DataFrame(columns=['publication_number', 'patent_title', 'appl_type', 
                           'positive_text'])
    total_positive_samples = 0
    total_publications = 0
    xml_text = html.unescape(open(filename, 'r').read())
    for patent in xml_text.split("<?xml version=\"1.0\" encoding=\"UTF-8\"?>"):

        if patent is None or patent == "":
            continue
        patent_text = patent
    
        bs = BeautifulSoup(patent, "lxml")
            
                #try:
                
        fwu_advantages = bs.find('heading',text='Advantageous Effects of Invention')
    
                #except Exception as e:
        
        
        
        
        
        
        
    ##### search for application or grant
        application = bs.find('us-patent-application')
        if application is None: # If no application, search for grant
            application = bs.find('us-patent-grant')
        
        title = "None"
            
    #### Search for its title
            
        try:
            title = application.find('invention-title').text
        except Exception as e:          
            #print("no title found")
            title = ""
                
        #print("patent is:", title)
            
            
    #### Search for publication number
        try:
            #publication_country = application.find('publication-reference').find('country').text
            #publication_doc_number = application.find('publication-reference').find('doc-number').text
            publication_kind = application.find('publication-reference').find('kind').text
            #publication_num = publication_country+publication_doc_number+publication_kind
        except Exception as e:
            #publication_num = ""
            publication_kind = ""
        
        
        try:
            publication_num = application['file'].split("-")[0]
        
        except Exception as e:
            publication_num = ""
        
        
        publication_num = publication_num+publication_kind
        #print(publication_num)  
    
        try:
            application_type = application.find('application-reference')['appl-type']
        
        except Exception as e:
            application_type =''
        #print(application_type)
            
        if publication_num:
            total_publications +=1

        if fwu_advantages:
            total_positive_samples +=1
        
            text = patent_text.splitlines()
            adv = []
            pos_text=""
            for i in text:
                #print(type(i))
                
                start = text.index(i)
                iteration = 1
                if ('>Advantageous Effects of Invention<'in i and iteration==1):
                    for j in range(20):
                        if '<p' in text[start+1]:
                            pos_text = text[start+1]
                            #print(pos_text)
                            adv.append(text[start+1])
                            start = start+1
                        else:
                            continue
                        
                    #print("patent is:", title)
                    #print(publication_num)
                    #print(application_type)
                    #print('-----------')
                    df = df.append({'publication_number':publication_num, 'patent_title': title,
                                'appl_type':application_type,
                               'positive_text':adv},ignore_index=True)
                
                    iteration=0
                
        #print('-----------')
        
        
        #else:
            #total_not_found +=1
        
    file = filename
    name = file.split('.')
    name =name[0]+'.csv'
    #name = name[-1].split('.')[0]    
    
    df.to_csv(name)  # for each week, there will be a seperate csv file created and later you can merge them
    print('total publications in',filename,total_publications )
    print('total positive samples in ', filename,total_positive_samples )
    df.shape
    print("------------")
    

In [ ]:
# call get_xml_data from here.

path = "/home/renuk/Documents/USPTO_BULK_DATA/patents/2020/"
all_files = glob.glob(os.path.join(path, "ipg*.xml"))
for filename in all_files:
    get_xml_data(filename)